In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re


df = pd.read_parquet('../habr_articles_parsed_final.parquet', engine='pyarrow')

/Users/georgij/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Сразу уберем из рассмотрения статьи, где нет контента, не будем их рассматривать

In [2]:
df = df[~df["Content"].isna()].reset_index(drop=True)

Начнем с того, что рассмотрим что именно представляют из себя таргетные теги

In [3]:
df['Tags'].str.split(', ').explode().value_counts()

Tags
javascript           6065
python               5863
программирование     4826
android              4621
разработка           4358
                     ... 
Radeon RX 5600 XT       1
Ryzen 7 4800U           1
Ryzen 7 4800H           1
udp streaming           1
некопипасть             1
Name: count, Length: 238850, dtype: int64

In [19]:
tags = df['Tags'].str.split(', ').explode()
unique_tags = tags.value_counts().index

Мы видим, что сейчас 238850 уникальных тегов, хотелось бы сильно сократить их количество

# Анализируем теги через семантическую модель

In [21]:
from sentence_transformers import SentenceTransformer

# Загружаем предобученную модель
model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
model = SentenceTransformer(model_name)

In [22]:
# Получаем эмбеддинги для тегов
vector_matrix = model.encode(tags.value_counts().index)

In [23]:
vector_matrix.shape

(238850, 512)

## DBScan Clustering

In [24]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

# Строим модель кластеризации, чтобы объединить теги по группам и убрать лишниз представителей
dbscan = DBSCAN(eps=0.1, min_samples=3, metric="cosine", n_jobs=10).fit(vector_matrix)
labels_dbscan = dbscan.labels_
unique_labels_dbscan = set(labels_dbscan)
print(len(unique_labels_dbscan))


9001


Посмотрим на группы

In [25]:
tags.value_counts().index[labels_dbscan == 1]

Index(['android', 'Android', 'android development', 'андроид',
       'разработка под android', 'android sdk', 'android os',
       'android разработка', 'андроид разработка', 'Андроид',
       ...
       'AndroidWear', 'Андроид разработка', 'androzic', 'Андроиды',
       'Androidandme', 'android-development', 'abdroid',
       'разработка по android', 'Android.Xiny', 'AndroidOS'],
      dtype='object', name='Tags', length=111)

In [26]:
mapping_tags_dbscan = dict(zip(unique_tags, labels_dbscan))

In [28]:
from tqdm import tqdm


unique_representation_dbscan = {label : unique_tags[labels_dbscan == label][0] for label in set(labels_dbscan)}
df["tags_classes_dbscan"] = None
for idx, item in tqdm(df.iterrows()):
    item_tags = item["Tags"].split(', ')
    class_tags = set()
    for item_tag in item_tags:
        if mapping_tags_dbscan[item_tag] > 0:
            class_tags.add(unique_representation_dbscan[mapping_tags_dbscan[item_tag]]) # уникальный представитель
        else:
            class_tags.add(item_tag)
        df.loc[idx, "tags_classes_dbscan"] = ", ".join(list(class_tags))

285498it [01:16, 3735.50it/s]


In [29]:
unique_tags_dbscan = df['tags_classes_dbscan'].str.split(', ').explode().value_counts()
unique_tags_dbscan

tags_classes_dbscan
искусственный интеллект     7526
google                      7293
машинное обучение           7085
android                     6920
linux                       6334
                            ... 
sqlite4                        1
профессиональное будущее       1
SQLite cusom function          1
приложение для Aндроид         1
некопипасть                    1
Name: count, Length: 186936, dtype: int64

Хотелось бы все равно оставить меньше тегов, желательно около 100

In [30]:
unique_tags_dbscan[unique_tags_dbscan > 1000]

tags_classes_dbscan
искусственный интеллект    7526
google                     7293
машинное обучение          7085
android                    6920
linux                      6334
                           ... 
юзабилити                  1020
наука                      1018
психология                 1016
twitter                    1013
selectel                   1013
Name: count, Length: 134, dtype: int64

Вот так звучит намного лучше. У нас будет минимум 1000 статей для каждого тега, будет проще обучить модель и предсказания будут стабильнее

In [31]:
unique_tags_dbscan_to_save = unique_tags_dbscan[unique_tags_dbscan > 1000].index

for idx, item in df.iterrows():
    item_tags_dbscan = item["tags_classes_dbscan"].split(', ')
    popular_tags = []
    for tag in item_tags_dbscan:
        if tag in unique_tags_dbscan_to_save:
            popular_tags.append(tag)
    df.loc[idx, "tags_classes_dbscan"] = ", ".join(popular_tags)

# Kmeans Clustering

Проделываем аналогичную процедуру с Kmeans

In [32]:
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

from sklearn import preprocessing

matrix_norm = preprocessing.normalize(vector_matrix) # нормализация нужна, чтобы было равносильно подсчету косинусного расстоянию

kmeans = KMeans(n_clusters=200, random_state=0).fit(matrix_norm)


In [33]:
labels_kmeans = kmeans.predict(matrix_norm)

In [43]:
tags.value_counts().index[labels_kmeans == 20]

Index(['астрономия', 'asterisk', 'nasa', 'Марс', 'lua', 'луна', 'спутники',
       'Луна', 'eclipse', 'НАСА',
       ...
       'Suborbital', 'SKY', 'itplanet', 'звездный воитель', 'follow the moon',
       'starfalltactics', 'астроциты', '100 000 звёзд', 'meteorite',
       'azure sphere'],
      dtype='object', name='Tags', length=1245)

In [38]:
mapping_tags_kmeans = dict(zip(unique_tags, labels_kmeans))

In [40]:
unique_representation = {label : unique_tags[labels_kmeans == label][0] for label in set(labels_kmeans)}
df["tags_classes_kmeans"] = None
for idx, item in tqdm(df.iterrows()):
    item_tags = item["Tags"].split(', ')
    class_tags = set()
    for item_tag in item_tags:
        if mapping_tags_kmeans[item_tag] > 0:
            class_tags.add(unique_representation[mapping_tags_kmeans[item_tag]]) # уникальный представитель
        else:
            class_tags.add(item_tag)
        df.loc[idx, "tags_classes_kmeans"] = ", ".join(list(class_tags))

285498it [01:17, 3664.13it/s]


In [41]:
unique_tags_kmeans = df['tags_classes_kmeans'].str.split(', ').explode().value_counts()
unique_tags_kmeans

tags_classes_kmeans
frontend             34333
ios                  25411
devops               25405
c                    25162
javascript           16547
                     ...  
темная жидкость          1
синтез днк               1
водяной лед              1
colonial pipeline        1
батарей                  1
Name: count, Length: 1926, dtype: int64

Удалось сократить количество, но хочется меньше

In [42]:
unique_tags_kmeans[unique_tags_kmeans > 1000]

tags_classes_kmeans
frontend         34333
ios              25411
devops           25405
c                25162
javascript       16547
                 ...  
unreal engine     1335
accessibility     1179
async             1150
функции           1070
часы              1008
Name: count, Length: 199, dtype: int64

Подходит, теперь уберем лишнее

In [44]:
unique_tags_kmeans_to_save = unique_tags_kmeans[unique_tags_kmeans > 1000].index

for idx, item in df.iterrows():
    item_tags_kmeans = item["tags_classes_kmeans"].split(', ')
    popular_tags = []
    for tag in item_tags_kmeans:
        if tag in unique_tags_kmeans_to_save:
            popular_tags.append(tag)
    df.loc[idx, "tags_classes_kmeans"] = ", ".join(popular_tags)

# Смотрим что получилось

In [45]:
df.head(50)

,Title,Author,Publication_date,Hubs,Tags,Content,Comments,Views,URL,Reading_time,Images_links,Individ/Company,Rating,Positive/Negative,Bookmarks_cnt,tags_classes_dbscan,tags_classes_kmeans
0,Лечение приступов лени,complex,2009-08-03 14:34:35+00:00,GTD,"лень, учись работать, самомотивация, мотивация",Пора лишать девственности свой бложик.\nТак ка...,67,6800,https://habr.com/ru/articles/66091/,2.0,,individual,4,positive,25.0,,"психология, монетизация, карьера, ml"
1,Как я работал по два часа в день,Konovalov,2009-07-30 13:50:03+00:00,GTD,"тайм-менеджмент, timemanagement, работа, эффек...",Когда я только перешёл от офисной работы к дом...,99,21000,https://habr.com/ru/articles/65783/,3.0,,individual,193,positive,114.0,работа,"бизнес, карьера, timeweb_статьи, менеджмент, п..."
2,Итоги первой недели,Irokez,2009-05-10 13:43:21+00:00,GTD,"тайм-менеджмент, пинарик, ворктрек","Итак, за прошедшую неделю хабраюзеры попробова...",29,1000,https://habr.com/ru/articles/59272/,1.0,https://habrastorage.org/r/w1560/getpro/megamo...,individual,15,positive,5.0,,"raspberry pi, менеджмент, devops"
3,Правильно поставленная задача время бережет,object,2009-05-05 00:05:11+00:00,GTD,"постановка задачи, планирование",Хочу поделиться своими мыслями по оптимизации ...,95,18000,https://habr.com/ru/articles/58889/,3.0,,individual,64,positive,75.0,,"планирование, ошибки"
4,ChatterBlocker или Вон из моей головы!,waitekk,2009-05-31 20:54:23+00:00,GTD,"chatterblocker, аудио","Как-то привык я к тому, что предметы вокруг ме...",56,5700,https://habr.com/ru/articles/60989/,1.0,http://www.picamatic.com/show/2009/06/01/01/01...,individual,28,positive,33.0,,"звук, конференция"
5,"Каким образом вы узнали то, что вы знаете?",ASPushkin,2009-07-02 20:08:04+00:00,GTD,"Joe Stagner, работа",Я получил это электронное письмо сегодня от Ма...,81,2500,https://habr.com/ru/articles/63367/,5.0,,individual,68,positive,165.0,работа,"Стив Джобс, карьера"
6,Организация рабочего времени с помощью цвета,popotam2,2009-07-15 20:24:31+00:00,GTD,"развитие, работоспособность, организация дел, ...",Предлагаю еще один вариант сделать себя более ...,13,3100,https://habr.com/ru/articles/64586/,1.0,http://www.neo-systems.ru/upload/image/calenda...,individual,1,positive,6.0,"разработка, бизнес","frontend, бизнес, разработка, карьера"
7,Enlarge your timus now!,Bobos,2009-06-09 13:52:55+00:00,GTD,"Организация личного времени, time management, ...","Привет, %username%.\nВчера я осознал, что снов...",130,1200,https://habr.com/ru/articles/61705/,3.0,,individual,67,positive,57.0,,"timeweb_статьи, freebsd"
8,Мегапланирование,Irokez,2009-07-06 15:43:39+00:00,GTD,"ворктрек, тайм-менеджмент, планирование, време...",В\nВорктреке\nс сегодняшнего дня появилась воз...,57,1100,https://habr.com/ru/articles/63706/,1.0,http://worktrek.ru/public/img/site/planning-1....,individual,30,positive,29.0,,"планирование, графика, менеджмент, devops"
9,Обмен услугами,4orever,2009-06-14 17:22:33+00:00,GTD,"работа, обмен услугами","Для всех, кто испытывает финансовые проблемы, ...",55,6600,https://habr.com/ru/articles/62053/,1.0,,individual,25,positive,8.0,работа,"сервисы, карьера"


In [46]:
df['tags_classes_kmeans'].str.split(', ').explode().value_counts()

tags_classes_kmeans
frontend         34333
ios              25411
devops           25405
c                25162
javascript       16547
                 ...  
accessibility     1179
async             1150
функции           1070
часы              1008
                    69
Name: count, Length: 200, dtype: int64

In [47]:
df['tags_classes_dbscan'].str.split(', ').explode().value_counts()

tags_classes_dbscan
                           107286
искусственный интеллект      7526
google                       7293
машинное обучение            7085
android                      6920
                            ...  
юзабилити                    1020
наука                        1018
психология                   1016
selectel                     1013
twitter                      1013
Name: count, Length: 135, dtype: int64

In [48]:
df[["Content", "URL", "tags_classes_dbscan", "tags_classes_kmeans"]].to_parquet("data_for_training.parquet")